<font size = 6 >Research track 02 | Assignment 2</font>

<font size = 4 color = 'Purple'>Student : </font>
<font size = 4 >Aicha Manar ABBAD</font>

<font size = 4 color = 'Purple'>Student ID: </font>
<font size = 4> s5565902</font>

--------------------------------------

# Notebook description

<font size = 4 > This Notebook servers as a user interface for the control a 2 wheeled mobile robot in a ROS environment. It also shows the graphical feedback of the state of the robot and its behaviour.</font> 

<font size = 4 > This Notebook is organized into 4 parts : </font>

   <font size = 4 > 1. Packages</font>
    
   <font size = 4 > 2. Classes</font>
   
   <font size = 4 > 3. Required functions</font>
   
   <font size = 4 > 4. Controlling robot motion using buttons</font>

## Node 'A'

<font size = 4 > Node 'A' implements an action client that allows the user to set a target goal or cancel it. The node also publishes and velocity as a costum message, which includes the values for (x, y, x_vel, y_vel) based the values published in "/odom" topic.</font>

<font size = 4 > Starting from (Node 'A') we aim to build a Jupyter Notebook to replace it. This is done by using widgets to let the user know: </font>

<font size = 4 >  - The position of the robot </font>

<font size = 4 >  - The goals that have been set and cancelled </font>

<font size = 4 >  - The distance to the closest obstacle </font>

--------------------------------------

<font size = 5 > __1. We start by importing the necessary packages :__ </font>

In [1]:
!pip3 install rospy rostopic rosparam

import rospy
import actionlib
import geometry_msgs.msg
import nav_msgs.msg
import time
import assignment_2_2022.msg

import matplotlib.pyplot as plt
import jupyros as jr
import ipywidgets as widgets
import numpy as np

In [2]:
from geometry_msgs.msg import PoseStamped, Twist
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import position, PlanningActionGoal, PlanningActionResult
from std_msgs.msg import String
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from matplotlib.patches import Arrow, Circle
from jupyros import ros3d
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, widgets
from IPython.display import display
from std_srvs.srv import Empty,EmptyResponse

from buttons import *

----------------

<font size = 5 > __2. Classes :__ </font>

<font size = 4 color = 'green'> __Visualiser()__ : </font>
<font size = 3 >This class is responsible for handling the visualization of a robot's position, orientation, and sensor data. </font>

<font size = 3 > "__ __init__ __ " : It creates two subplots for plotting the robot's initial position (odom), the goal position (goal), and laser scan data (laser_scan) </font>

<font size = 3 > "__odom_callback__" : A callback function that receives data about the robot's position and velocity. It appends the x and y coordinates to the corresponding lists and stores the velocity values. </font>

<font size = 3 > "__goal_callback__" : A callback function that receives the goal position. It appends the new x and y coordinates to the corresponding lists.</font>

<font size = 3 > "__laser_callback__" : A callback function  that receives laser scan data. It stores the laser scan data in a list and returns it.</font>

<font size = 3 > "__plot_update_1__" and  "__plot_update_2__": Are ccalled to update the subplots with the latest robot position and goal position as well as the latest laser scan data.</font>

In [3]:
class Visualiser:
    
    def __init__(self):
        
        self.figure_1, self.ax1 = plt.subplots()
        self.ax1.set_title("Odometry")
        self.ax1.set_xlabel("X - Coordinates")
        self.ax1.set_ylabel("Y - Coordinates")
        self.ax1.set_xlim(-12,12)
        self.ax1.set_ylim(-12,12)
        self.odom = self.ax1.plot([],[],'ro', label = 'Robot position')
        self.goal = self.ax1.plot([],[],'b*', label = 'Goal')
        plt.legend(facecolor = '#faebd7', fontsize = 10)
        
        self.figure_2, self.ax2 = plt.subplots()
        self.ax2.set_title("Laser")
        self.ax2.set_xlabel("X - Coordinates")
        self.ax2.set_ylabel("Y - Coordinates")
        self.ax2.set_xlim(-12,12)
        self.ax2.set_ylim(-12,12)
        self.laser_scan = self.ax2.plot([],[],'o', label = "Laser scans")
        plt.legend(facecolor = '#faebd7', fontsize = 10)
    
        self.x_odom = []
        self.y_odom = []
        self.x_goal = []
        self.y_goal = []
        self.laser = []
        self.plot_init()
        
        
    def plot_init(self):
        
        return self.odom, self.goal, self.laser_scan
        
                
    def odom_callback(self,data):
        self.x_odom.append(data.pose.pose.position.x)
        self.y_odom.append(data.pose.pose.position.y)
        self.x_vel = data.twist.twist.linear.x
        self.y_vel = data.twist.twist.linear.y
    
    def goal_callback(self,position):
        self.x_goal.clear()
        self.y_goal.clear()
        self.x_goal.append(position.goal.target_pose.pose.position.x)
        self.y_goal.append(position.goal.target_pose.pose.position.y)
        
    def laser_callback(self,msg):
        self.laser = list(msg.ranges)
        return self.laser
    
        
    def plot_update_1(self,frame):
        self.odom.set_data(self.x_odom,self.y_odom)
        self.goal.set_data(self.x_goal,self.y_goal)
        return self.odom, self.goal
    
    
    def plot_update_2(self,frame):
        self.laser_scan.set_data(list(np.linspace(0,720,720)),self.laser)
        return self.laser_scan

<font size = 4 color = 'green'> __Goals()__ : </font>
<font size = 3 >This class is responsible for tracking and visualizing the progress of goals being reached or cancelled. </font>

<font size = 3 > "__ __init__ __" : It creates a new figure to show the initial count of goals reached and goals cancelled.  </font>

<font size = 3 > "__reached_callback__" : It is a callback function that receives data about the status of a goal and increase the number of reached / cancelled goals.  </font>

<font size = 3 > "__plot_update_3__" : It is responsible for updating the plot with the latest counts of reached and cancelled goals.  </font>

In [4]:
class Goals:
    
    def __init__(self):
        
        self.goal_reached = 0
        self.goal_cancelled = 0

        self.figure = plt.figure()
        
        plots = plt.rcParams["figure.autolayout"] = True
        plt.bar(["Goals reached", "Goals cancelled"],[self.goal_reached,self.goal_cancelled], color =['green','red'])
        
    def reached_callback(self, data):
        
        if(data.status.status == 3):
            self.goal_reached = self.goal_reached + 1
            
        elif(data.status.status == 2):
            self.goal_cancelled = self.goal_cancelled + 1
        
        
    def plot_update_3(self,frame):
        plots = plt.bar(["Goal reached", "Goal ancelled"],[self.goal_reached,self.goal_cancelled], color =['green','red'])
        
        

<font size = 4 color = 'green'> __Distance()__ : </font>
<font size = 3 >  computes the distance between the current position and a desired position, and stores the computed distance in an instance variable. </font>

<font size = 3 > "__ __init__ __" : Initialize the value of distance, velocity, starting and ending time to zero. </font>

<font size = 3 > "__sub_callback__" : A callback faction which is called to compute the distance and velocity.</font>

<font size = 3 > "__compute_distance__" : A function that is defined to calculate the distance based on the data received. It retrieves the desired position coordinates from ROS parameters, gets the current position coordinates from the data, and then computes the distance between the two points using the distance formula. The calculated distance is stored, and it is also printed to the console.</font>

In [5]:
class Distance:
    
    def __init__(self):
        
        self.distance =  0.0
        self.begin_time = 0.0
        self.end_time = 0.0
        self.velocity = 0.0
    
    
    def sub_callback(self, data):
        self.compute_distance()
        self.compute_velocity()
    
    
    def compute_distance(self,data):

        self.x_pos = rospy.get_param("/des_pos_x")
        self.y_pos = rospy.get_param("/des_pos_y")

        self.x = data.x_pos
        self.y = data.y_pos
        self.begin_time = rospy.Time.now()

        self.x_dist = self.x_pos - self.x
        self.y_dist =self.y_pos - self.y
        self.distance = math.sqrt((self.x_dist**2) + (self.y_dist**2)) 
        print("The distance to the goal : ")
        print(self.distance)
        

---------------

<font size = 5 > __3. Required Functions :__ </font>

<font size = 3 > "__start_simulation__" : This function is called when the "start" button is clicked. It changes the style of the "start" button to indicate that it is active (using the 'success' style), and clears the style of the "stop" button. </font>

<font size = 3 > "__stop_simulation__" : This function is called when the "stop" button is clicked. It changes the style of the "stop" button to indicate that it is active (using the 'danger' style), and clears the style of the "start" button.</font>

<font size = 3 > "__move_forward__" : This function is called when the "up" button is clicked. It publishes a velocity message with a positive linear velocity to move forward. It also changes the style of the "up" button to indicate that it is active (using the 'info' style), and clears the styles of the other movement buttons.</font>

<font size = 3 > "__move_backward__" : This function is called when the "down" button is clicked. It publishes a velocity message with a negative linear velocity to move backward. It changes the style of the "down" button to indicate that it is active (using the 'info' style), and clears the styles of the other movement buttons.</font>

<font size = 3 > "__turn_left__" : This function is called when the "left" button is clicked. It publishes a velocity message with a negative angular velocity to turn left. It changes the style of the "left" button to indicate that it is active (using the 'info' style), and clears the styles of the other movement buttons.</font>

<font size = 3 > "__turn_right__" : This function is called when the "right" button is clicked. It publishes a velocity message with a positive angular velocity to turn right. It changes the style of the "right" button to indicate that it is active (using the 'info' style), and clears the styles of the other movement buttons.</font>

<font size = 3 > "__stop_movement__" : This function is called when the "stops" button is clicked. It publishes a velocity message with zero linear and angular velocities to stop the movement. It changes the style of the "stops" button to indicate that it is active (using the 'danger' style), and clears the styles of the other movement buttons.</font>

<font size = 3 > "__button_event__" : This function sets up the event handling for the buttons. It assigns the appropriate functions to the __on_click__ events of each button, so that when they are clicked, the corresponding functions are called.</font>

In [6]:
def start_simulation(a):
    #ui_client("start")
    start.button_style = 'success'
    stop.button_style = ''
    
    
def stop_simulation(b):
    #ui_client("stop")
    start.button_style = ''
    stop.button_style = 'danger'  

def move_forward(a):
    global pub
    lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*0.5
    pub.publish(vel)
    up.button_style = 'info'
    down.button_style = ''
    left.button_style = ''
    right.button_style = ''
    stops.button_style = ''
    
    
def move_backward(a):
    global pub
    lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*(-0.5)
    pub.publish(vel)
    up.button_style = ''
    down.button_style = 'info'
    left.button_style = ''
    right.button_style = ''
    #stops.button_style = ''

def turn_left(a):
    global pub
    av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(-0.5)
    pub.publish(vel)
    up.button_style = ''
    down.button_style = ''
    left.button_style = 'info'
    right.button_style = ''
    stop.button_style = ''


def turn_right(a):
    global pub
    av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(0.5)
    pub.publish(vel)
    up.button_style = ''
    down.button_style = ''
    left.button_style = ''
    right.button_style = 'info'
    stops.button_style = ''

    
def stop_movement(a):
    global pub
    vel = Twist()
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    up.button_style = ''
    down.button_style = ''
    left.button_style = ''
    right.button_style = ''
    stops.button_style = 'danger'

    
def button_event():
    
    start.on_click(start_simulation)
    stop.on_click(stop_simulation)
    up.on_click(move_forward)
    down.on_click(move_backward)  
    left.on_click(turn_left)
    right.on_click(turn_right)  
    stops.on_click(stop_movement)
    

------------

<font size = 3 >The __main()__ function is then defined, which serves as the entry point for the program. It initializes a ROS node, sets up ROS subscribers and publishers, creates and animates plots using matplotlib, and defines callback functions for handling received data.</font>



In [7]:
%matplotlib widget

def main():
    visualiser = Visualiser()
    
    # Publish to "/cmd_vel"
    global pub
    pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
    
    # Subscribe to "/odom"
    sub1 = rospy.Subscriber("/odom", Odometry,visualiser.odom_callback)
    # Subscribe to "/reaching_goal/goal"
    sub2 = rospy.Subscriber("/reaching_goal/goal", PlanningActionGoal, visualiser.goal_callback)
    # Subscribe to "/scan"
    sub3 = rospy.Subscriber("/scan", LaserScan, visualiser.laser_callback)
    
    # Plot and Animate the plots
    FuncAnimation(visualiser.figure_1, visualiser.plot_update_1, init_func = visualiser.plot_init, blit = True)
    FuncAnimation(visualiser.figure_2, visualiser.plot_update_2, init_func = visualiser.plot_init, blit = True)
    plt.show(block=True)
    
    
    goals = Goals()
    rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, goals.reached_callback)
    FuncAnimation(goals.figure, goals.plot_update_3, interval = 100)
    plt.show()

if __name__ == "__main__":
    
    rospy.init_node("node_a")
    button_event()
    main()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-------------

<font size = 5 > __4. Controlling the robot motion using buttons__<font>

<font size = 4 >In this part we try to control the motion of the robot (Up, Down, Left, Right, Stop) using widget buttons.<font>

<font size = 4 color = 'green'> __Start__<font> <font size = 4 > and <font> 
<font size = 4 color = 'RED'> __Stop__<font>

In [8]:
HBox([start,stop])

<font size = 4 color = 'blue'> __Up__<font> <font size = 4 > and <font> 
<font size = 4 color = 'blue'> __Down__<font>

In [9]:
HBox([up,down])

<font size = 4 color = 'blue'> __Right__<font> <font size = 4 > and <font> 
<font size = 4 color = 'blue'> __Left__<font>

In [10]:
HBox([left,right])

<font size = 4 color = 'RED'> __Stop__<font>

In [11]:
display(stops)

Button(button_style='danger', icon='stop', layout=Layout(grid_area='stop', height='50px', width='auto'), style…